In [1]:
# pretrained substitute
# Encodec as a replacement for SoundStream, and MERT as a replacement for w2v-BERT.
# idea from https://github.com/zhvng/open-musiclm

In [1]:
!rm -rf audiolm_pytorch/ setup.py audiolm-pytorch.zip # clean out any old stuff floating around

!pip install torch datasets

# download audiolm_pytorch manually so i can inject print statements
# !pip uninstall -y audiolm_pytorch

# raise AssertionError("don't forget to upload the customized version of audiolm_pytorch with print statements")
# !zip -r audiolm_pytorch.zip audiolm_pytorch/

import urllib.request
import os
import zipfile
if not os.path.isfile("audiolm-pytorch.zip"):
  urllib.request.urlretrieve("https://github.com/LWProgramming/audiolm-pytorch/archive/refs/heads/personal_hacks.zip", "audiolm-pytorch.zip")
if not os.path.isdir("audiolm-pytorch"):
  with zipfile.ZipFile("audiolm-pytorch.zip", 'r') as zip_ref:
    zip_ref.extractall("audiolm-pytorch")
!mv audiolm-pytorch/audiolm-pytorch-personal_hacks/audiolm_pytorch .

# install necessary files for patched audiolm-pytorch
!mv audiolm-pytorch/audiolm-pytorch-personal_hacks/setup.py .
!pip install . # install requirements from the patched audiolm-pytorch dir
!rm -rf audiolm-pytorch # not the one with underscore which is the actual library

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content
  Preparing metadata (setup.py) ... done
  Created wheel for audiolm-pytorch: filename=audiolm_pytorch-0.22.2-py3-none-any.whl size=31138 sha256=61861e058ce54dcd8d516719ea35e67587c2907eac006fdce69d3a4609a43f47
  Stored in directory: /tmp/pip-ephem-wheel-cache-r001hiu9/wheels/fd/3e/a8/f6cc18082b087ea481047dfbb1056db08877ac1382dcc607b7
Successfully built audiolm-pytorch
  Attempting uninstall: audiolm-pytorch
    Found existing installation: audiolm-pytorch 0.22.2
    Uninstalling audiolm-pytorch-0.22.2:
      Successfully uninstalled audiolm-pytorch-0.22.2


In [3]:
# # semantic- MERT
# # https://huggingface.co/m-a-p/MERT-v0
# # MERT-v0 is a completely unsupervised model trained on 1000 hour music audios.

# from transformers import Wav2Vec2Processor, HubertModel
# import torch
# from torch import nn
# from datasets import load_dataset

# # load demo audio and set processor
# dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
# dataset = dataset.sort("id")
# sampling_rate = dataset.features["audio"].sampling_rate
# processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")

# # loading our model weights
# model = HubertModel.from_pretrained("m-a-p/MERT-v0")

# # audio file is decoded on the fly
# inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
# with torch.no_grad():
#     outputs = model(**inputs, output_hidden_states=True)

# # take a look at the output shape, there are 13 layers of representation
# # each layer performs differently in different downstream tasks, you should choose empirically
# all_layer_hidden_states = torch.stack(outputs.hidden_states).squeeze()
# print(all_layer_hidden_states.shape) # [13 layer, 292 timestep, 768 feature_dim]

# # # for utterance level classification tasks, you can simply reduce the representation in time
# # time_reduced_hidden_states = all_layer_hidden_states.mean(-2)
# # print(time_reduced_hidden_states.shape) # [13, 768]

# # # you can even use a learnable weighted average representation
# # aggregator = nn.Conv1d(in_channels=13, out_channels=1, kernel_size=1)
# # weighted_avg_hidden_states = aggregator(time_reduced_hidden_states.unsqueeze(0)).squeeze()
# # print(weighted_avg_hidden_states.shape) # [768]


In [2]:
# original semantic transformer
import torch
from audiolm_pytorch import HubertWithKmeans, SemanticTransformer, SemanticTransformerTrainer
import os
import urllib

# hubert checkpoints can be downloaded at
# https://github.com/facebookresearch/fairseq/tree/main/examples/hubert

hubert_ckpt = 'hubert/hubert_base_ls960.pt'
hubert_quantizer = f'hubert/hubert_base_ls960_L9_km500.bin' # listed in row "HuBERT Base (~95M params)", column Quantizer
if not os.path.isdir("hubert"):
  os.makedirs("hubert")
if not os.path.isfile(hubert_ckpt):
  hubert_ckpt_download = f"https://dl.fbaipublicfiles.com/{hubert_ckpt}"
  urllib.request.urlretrieve(hubert_ckpt_download, f"./{hubert_ckpt}")
if not os.path.isfile(hubert_quantizer):
  hubert_quantizer_download = f"https://dl.fbaipublicfiles.com/{hubert_quantizer}"
  urllib.request.urlretrieve(hubert_quantizer_download, f"./{hubert_quantizer}")

wav2vec = HubertWithKmeans(
    checkpoint_path = './hubert/hubert_base_ls960.pt',
    kmeans_path = './hubert/hubert_base_ls960_L9_km500.bin'
)

semantic_transformer = SemanticTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    dim = 1024,
    depth = 6
).cuda()

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator MiniBatchKMeans from version 0.24.0 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
# import wave
# import struct

# # dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
# # dataset[0]["audio"]["array"]
# sampling_rate = dataset.features["audio"].sampling_rate

# def save_wav(file_name, audio, sample_rate=sampling_rate):
#   # Open up a wav file
#   wav_file=wave.open(file_name,"w")
#   # wav params
#   nchannels = 1
#   sampwidth = 2
#   # 44100 is the industry standard sample rate - CD quality.  If you need to
#   # save on file size you can adjust it downwards. The stanard for low quality
#   # is 8000 or 8kHz.
#   nframes = len(audio)
#   comptype = "NONE"
#   compname = "not compressed"
#   wav_file.setparams((nchannels, sampwidth, sample_rate, nframes, comptype, compname))
#   # WAV files here are using short, 16 bit, signed integers for the 
#   # sample size.  So we multiply the floating point data we have by 32767, the
#   # maximum value for a short integer.  NOTE: It is theortically possible to
#   # use the floating point -1.0 to 1.0 data directly in a WAV file but not
#   # obvious how to do that using the wave module in python.
#   for sample in audio:
#     wav_file.writeframes(struct.pack('h', int( sample * 32767.0 )))
#   wav_file.close()
#   return
# save_wav("test.wav", dataset[1]["audio"]["array"])

In [4]:
from audiolm_pytorch import SemanticTransformerWrapper
import numpy as np

# in case not already loaded
from datasets import load_dataset
# load demo audio and set processor
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")

batch_size = 1
# sample data[1] is 77040 samples at 16kHz sampling rate, 93680 for dataset[0]
# just reshape it here so batch size for prime_wave is effectively 1
samples = np.array([dataset[0]["audio"]["array"]])
prime_wave = torch.tensor(samples).reshape(len(samples), len(samples[0])).cuda() # reshape from samples to batch x num_samples
print(prime_wave.shape)
# raise AssertionError(prime_wave.shape)
max_length = 2048
semantic = SemanticTransformerWrapper(
            wav2vec = wav2vec,
            transformer = semantic_transformer,
            audio_conditioner = None,
            unique_consecutive = True
        ).cuda()
semantic_tokens = semantic.generate(
            text_embeds = None, # no text, it's not musicLM
            batch_size = batch_size,
            prime_wave = prime_wave,
            max_length = max_length
        )
semantic_tokens.shape

torch.Size([1, 93680])
embed.keys(): dict_keys(['x', 'padding_mask', 'features'])
embed['x'] shape: torch.Size([1, 292, 768]), embed['features'].shape: torch.Size([1, 292, 768])
wav_input shape: torch.Size([1, 93680]), embed shape: torch.Size([292, 768]), packed_shape: [torch.Size([1, 292])]
codebook_indices before unpacking: torch.Size([292])
codebook_indices after unpacking: torch.Size([1, 292])
ids.shape: torch.Size([1, 292]) and prime_wave True


generating semantic:  18%|█▊        | 345/1895 [00:06<00:30, 51.52it/s]

before masking eos, sample_semantic_ids.shape: torch.Size([1, 499])


torch.Size([1, 499])

In [5]:
# ?semantic.wav2vec
# torch.tensor(dataset[1]["audio"]["array"]).cuda().device
# semantic_tokens.shape
# semantic_tokens[:,-1] # exactly the EOS which is set as w2v.codebook_size == 500 in the case of k-means with k=500
# semantic_tokens[:, 0]
semantic_tokens[:, -1]
# semantic.pad_id

tensor([500], device='cuda:0')

In [8]:
# dataset[1]["audio"]["array"].shape
# # len(dataset[1]["audio"]["array"]) # 77040
# # dataset.features["audio"].sampling_rate # 16000
# # so 4.815 seconds of audio

In [9]:
torch.empty((batch_size, 0), dtype = torch.long)

tensor([], size=(1, 0), dtype=torch.int64)

In [10]:
semantic_transformer.eos_id

500